In [1]:
import sys
sys.path.insert(0, "/home/aisummer/mikhail_workspace/nlp_service")
import pandas as pd 
import json
import os
import time
import tqdm

from src.DocumentsParser.utils import DBS_DIR_DENSE_VECTORDB_NAME, DBS_DIR_SPARSE_VECTORDB_NAME
from src.evaluation_metrics import ReaderMetrics
from src.DocumentsSummarizer.Summarizer import SummarizerModule
from src.DocumentsSummarizer.utils import SummarizerConfig
from src.utils import DialogueState
from src.logger import Logger
from langchain_core.documents.base import Document

### 1) Создаём объекты

In [2]:
logger = Logger(False)
log = logger.get_logger(__name__)

config = SummarizerConfig.load('/home/aisummer/ssemenov_workspace/nlp-service/config.yaml')
reader = SummarizerModule(config, log)


llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/aisummer/nlp_models/model_llm/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32 

llama_model_loader: - kv  17:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  18:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  19:                tokenizer.ggml.eos_token_id u32              = 128009
llama_model_loader: - kv  20:                    tokenizer.chat_template str              = {% set loop_messages = messages %}{% ...
llama_model_loader: - kv  21:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q4_0:  225 tensors
llama_model_loader: - type q6_K:    1 tensors
llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.8000 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_met

### 2) Указываем инфо для данных

In [3]:
BENCHMARKS_INFO = {'sberquad': 'v1', 'squadv2': 'v1'}
MODEL_PATH = '/home/aisummer/nlp_models/model_llm/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa'
BENCHES_SIZE = 500
SAVE_LOGFILE = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/logs/trial5.json'
SAVE_CSVFILE_1 = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/csv_log/sberquad_trial5_.csv'
SAVE_CSVFILE_2 = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/csv_log/squadv2_trial5.csv'

banchmark_paths = {}
for name, version in BENCHMARKS_INFO.items():
    banchmark_paths[name] = {
        'table': f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/tables/{version}/benchmark.csv",
        'dense_db': f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/dbs/{version}/{DBS_DIR_DENSE_VECTORDB_NAME}",
        'sparse_db':  f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/dbs/{version}/{DBS_DIR_SPARSE_VECTORDB_NAME}"
    }

### 3) Загрузка датасета

In [4]:
# загрузить benchmark-датасет
benchmarks_df = {}
for name, bench_path in banchmark_paths.items():
    benchmarks_df[name] = pd.read_csv(banchmark_paths[name]['table'], sep=';')

# инициализировать класс с метриками
base_dir = '/home/aisummer/ssemenov_workspace/nlp-service'
metrics = ReaderMetrics(base_dir)

# logging
logger = Logger(False)
log = logger.get_logger(__name__)

Loading Meteor...
Loading ExactMatch


In [5]:
benchmarks_df['sberquad']

,question,answer,chunk_ids,contexts
0,чем представлены органические остатки?,известковыми выделениями сине-зелёных водорослей,[4974587056],['В протерозойских отложениях органические ост...
1,что найдено в кремнистых сланцах железорудной ...,"нитевидные водоросли, грибные нити",[4974587056],['В протерозойских отложениях органические ост...
2,что встречается в протерозойских отложениях?,органические остатки,[4974587056],['В протерозойских отложениях органические ост...
3,что относится к числу древнейших растительных ...,скопления графито-углистого вещества,[4974587056],['В протерозойских отложениях органические ост...
4,как образовалось графито-углистое вещество?,в результате разложения Corycium enigmaticum,[4974587056],['В протерозойских отложениях органические ост...
...,...,...,...,...
74251,Где используется классическая трёхуровневая си...,в рубиновом лазере.,[1898313107],['Классическая трёхуровневая система накачки р...
74252,какой инвестиционный банк входил в состав Amer...,Lehman Brothers,[4929086505],['Первая платёжная карта American Express появ...
74253,С каким альбомом был созвучен альбом Дэвида Бо...,Low,[1303094225],['Следующий альбом Heroes был во многом созвуч...
74254,В каком техасском оркестре выступал гитарист Л...,Light Crust Doughboys,[934655466],"['Одним из тех, на кого игра Данна произвела н..."


In [6]:
benchmarks_df['squadv2']

,question,answer,chunk_ids
0,When did Beyonce start becoming popular?,in the late 1990s,[621329309]
1,What areas did Beyonce compete in when she was...,singing and dancing,[621329309]
2,When did Beyonce leave Destiny's Child and bec...,2003,[621329309]
3,In what city and state did Beyonce grow up?,"Houston, Texas",[621329309]
4,In which decade did Beyonce become famous?,late 1990s,[621329309]
...,...,...,...
141964,What is the seldom used force unit equal to on...,sthène,[2990931136]
141965,What does not have a metric counterpart?,NaN,[2990931136]
141966,What is the force exerted by standard gravity ...,NaN,[2990931136]
141967,What force leads to a commonly used unit of mass?,NaN,[2990931136]


### 4) Предскажем ответы по вопросам из датасетов 

In [7]:
def get_relevant_ans(df: dict, reader: SummarizerModule, cnt: int, contexts: bool) -> list:
    relevant_ans = []
    questions = []
    context = []
    question = df['question']
    
    for i in tqdm.tqdm(range(cnt)):

        questions.append(question[i])

        if contexts:
            context.append(df['contexts'][i].strip('[]').strip("'"))
            contexts = df['contexts'][i].strip('[]').strip("'")
        else:
            contexts = ''

        relevant_chunks = [contexts.strip('[]').strip("'")]
        state = DialogueState(
            query=question[i],
            base_relevant_docs = [Document(page_content=chunk) for chunk in relevant_chunks])

        
        output = reader.create_answer(state)
        result=''
        for item in output:
            try:
                result += item['choices'][0]['delta']['content']
            except Exception:
                continue
        relevant_ans.append(result)
    return questions, relevant_ans, context

### sberquad 

In [8]:
start_time_sberquad = time.time()

cnt = BENCHES_SIZE
questions_sberquad, predicted_sberquad, context_sberquad = get_relevant_ans(benchmarks_df['sberquad'], reader, cnt, True)
targets_sberquad = benchmarks_df['sberquad']['answer'].to_list()[:cnt]

time_execute_sberquad = time.time() - start_time_sberquad

q_and_a_sberquad = {
    'question': questions_sberquad,
    'predicted_answer': predicted_sberquad,
    'target': targets_sberquad,
    'context': context_sberquad
    }

  0%|          | 0/500 [00:00<?, ?it/s]


llama_print_timings:        load time =    4946.73 ms
llama_print_timings:      sample time =      26.00 ms /    48 runs   (    0.54 ms per token,  1846.44 tokens per second)
llama_print_timings: prompt eval time =    4946.50 ms /   275 tokens (   17.99 ms per token,    55.59 tokens per second)
llama_print_timings:        eval time =    4899.06 ms /    47 runs   (  104.24 ms per token,     9.59 tokens per second)
llama_print_timings:       total time =    9902.34 ms /   322 tokens
  0%|          | 1/500 [00:09<1:22:23,  9.91s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    4946.73 ms
llama_print_timings:      sample time =      16.39 ms /    30 runs   (    0.55 ms per token,  1830.61 tokens per second)
llama_print_timings: prompt eval time =     476.44 ms /    26 tokens (   18.32 ms per token,    54.57 tokens per second)
llama_print_timings:        eval time =    3079.28 ms /    29 runs   (  106.18 ms per token,     9.42 tokens per second)
llama_print_t

In [9]:
df_sberquad = pd.DataFrame(q_and_a_sberquad)
df_sberquad.to_csv(SAVE_CSVFILE_1)
df_sberquad.head(10)

,question,predicted_answer,target,context
0,чем представлены органические остатки?,Органические остатки в протерозойских отложени...,известковыми выделениями сине-зелёных водорослей,В протерозойских отложениях органические остат...
1,что найдено в кремнистых сланцах железорудной ...,"Нитевидные водоросли, грибные нити и формы, бл...","нитевидные водоросли, грибные нити",В протерозойских отложениях органические остат...
2,что встречается в протерозойских отложениях?,В протерозойских отложениях органические остат...,органические остатки,В протерозойских отложениях органические остат...
3,что относится к числу древнейших растительных ...,"Скопления графито-углистого вещества, образова...",скопления графито-углистого вещества,В протерозойских отложениях органические остат...
4,как образовалось графито-углистое вещество?,Образование графито-углистого вещества в Проте...,в результате разложения Corycium enigmaticum,В протерозойских отложениях органические остат...
5,Куда поступают остатки непереваренной пищи?,Остатки непереваренной пищи поступают в толсты...,в толстый кишечник,Кишечник млекопитающего подразделяется на тонк...
6,Каким процессам подвергается непереваренная пи...,Бродильным процессам с участием эндосимбионтов...,бродильным процессам с участием эндосимбионтов,Кишечник млекопитающего подразделяется на тонк...
7,В каком кишечнике переваривается основная част...,Основная часть пищи переваривается в тонком ки...,в тонком кишечнике,Кишечник млекопитающего подразделяется на тонк...
8,Какие виды анатомических сегментов выделяют в ...,У млекопитающих выделяют 4 анатомических сегме...,тонкий и толстый,Кишечник млекопитающего подразделяется на тонк...
9,Для каких видов млекопитающих особо важна слеп...,Слепая кишка играет особенно важную роль у вид...,"Для видов, кормящихся грубой растительной пищей",Кишечник млекопитающего подразделяется на тонк...


### squadv2

In [10]:
start_time_squadv2 = time.time()

questions_squadv2, predicted_squadv2, _ = get_relevant_ans(benchmarks_df['squadv2'], reader, cnt, False)
targets_squadv2 = benchmarks_df['squadv2']['answer'].to_list()[:cnt]

time_execute_squadv2 = time.time() - start_time_squadv2

q_and_a_squadv2 = {
    'question': questions_squadv2,
    'predicted_answer': predicted_squadv2,
    'target': targets_squadv2
}

  0%|          | 0/500 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    4946.73 ms
llama_print_timings:      sample time =      36.62 ms /    69 runs   (    0.53 ms per token,  1883.96 tokens per second)
llama_print_timings: prompt eval time =     349.94 ms /    18 tokens (   19.44 ms per token,    51.44 tokens per second)
llama_print_timings:        eval time =    7237.98 ms /    68 runs   (  106.44 ms per token,     9.39 tokens per second)
llama_print_timings:       total time =    7669.36 ms /    86 tokens
  0%|          | 1/500 [00:07<1:03:47,  7.67s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    4946.73 ms
llama_print_timings:      sample time =      10.95 ms /    21 runs   (    0.52 ms per token,  1917.11 tokens per second)
llama_print_timings: prompt eval time =     350.23 ms /    18 tokens (   19.46 ms per token,    51.40 tokens per second)
llama_print_timings:        eval time =    2193.24 ms /    20 

In [11]:
df_squadv2 = pd.DataFrame(q_and_a_squadv2)
df_squadv2.to_csv(SAVE_CSVFILE_2)
df_squadv2.head(10)

,question,predicted_answer,target
0,When did Beyonce start becoming popular?,Beyoncé started gaining popularity in the late...,in the late 1990s
1,What areas did Beyonce compete in when she was...,"As a child, Beyoncé competed in dance and musi...",singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,Beyoncé left Destiny's Child in 2000 to pursue...,2003
3,In what city and state did Beyonce grow up?,"Beyoncé grew up in Houston, Texas.","Houston, Texas"
4,In which decade did Beyonce become famous?,The 2000s!,late 1990s
5,In what R&B group was she the lead singer?,En Vogue!,Destiny's Child
6,What album made her a worldwide known artist?,The album that made Lady Gaga a worldwide know...,Dangerously in Love
7,Who managed the Destiny's Child group?,Mathew Knowles.,Mathew Knowles
8,When did Beyoncé rise to fame?,Beyoncé rose to fame in the late 1990s as a me...,late 1990s
9,What role did Beyoncé have in Destiny's Child?,Beyoncé was the lead vocalist and one of the f...,lead singer


### 6) Посчитаем метрики

In [12]:
benchmarks_score = {}
def metric(predicted: list[str], targets: list[str], name:str) -> None:
    
    length = len(predicted)

    print('Dataset', name, '\n')
    rouge_score = metrics.rouge(predicted, targets[:length])
    print("rouge: ", rouge_score)

    bleu_score = metrics.bleu(predicted, targets[:length])
    print("bleu: ", bleu_score)

    meteor = metrics.meteor(predicted, targets[:length])
    print("meteor: ", meteor)

    exact_match = metrics.exact_match(predicted, targets[:length])
    print("exact_match: ", exact_match, '\n')

    # посчитать метрики 
    score = {
        'rouge': float(rouge_score),
        'bleu': float(bleu_score),
        'meteor': float(meteor),
        'exact_match': float(exact_match)
    }

    return score



score = metric(predicted_sberquad, targets_sberquad,'sberquad')
benchmarks_score['sberquad'] = score

score = metric(predicted_squadv2, targets_squadv2, 'squadv2')
benchmarks_score['squadv2'] = score




Dataset sberquad 

rouge:  0.2015
bleu:  0.15456
meteor:  0.54695
exact_match:  0.02 

Dataset squadv2 

rouge:  0.17474
bleu:  0.02093
meteor:  0.1766
exact_match:  0.0 



In [13]:

# сохранить результат
if os.path.exists(SAVE_LOGFILE):
    print("Файл существует!")
    raise ValueError

log_data = {'info': BENCHMARKS_INFO, 'time_execute_sberquad': float(time_execute_sberquad), 'time_execute_squadv2': float(time_execute_squadv2),
            'hyperp': {
                'model_path': MODEL_PATH, 
                'tech_params': config.tech_config.__dict__, 'hyper_params': config.strat_config.__dict__,'benchmark_sizes': BENCHES_SIZE},
                'scores': benchmarks_score}

with open(SAVE_LOGFILE, 'w', encoding='utf-8') as fd:
    fd.write(json.dumps(log_data, indent=1))